# Adecuar el ambiente de desarrollo

## 1. Habilitar google drive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

## Instalar librerias necesarias

In [1]:
# instalar librería para visualizar el progreso de ejecución una tarea en background
pip install pyprind

##paso 1.3 Crear funciones globales

In [0]:
#función para ver avance de proceso en background
def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = progress_size / (1024.**2 * duration)
    percent = count * block_size * 100. / total_size
    sys.stdout.write("\r%d%% | %d MB | %.2f MB/s | %d segundos transcurrido" %
                    (percent, progress_size / (1024.**2), speed, duration))
    sys.stdout.flush()

##paso 1.3 Crear funciones globales

In [0]:
#funcion para descargar cada uno de los datasets
def download_dataset(Source, Target, drivepath):
  #si no existe el directorio debe crearlo
  if not os.path.exists(drivepath):  
    os.mkdir(drivepath)
  #si no existe el archivo descarguelo de lo contrario no
  if not os.path.isfile(drivepath+'/'+Target):  
  #verificar que la versión de python
    if (sys.version_info < (3, 0)):
      import urllib
      urllib.urlretrieve(Source, drivepath+'/'+Target, reporthook)
    else:
      import urllib.request
      urllib.request.urlretrieve(Source, drivepath+'/'+Target, reporthook)
  else:
    print("The file "+drivepath+'/'+Target+" is downloaded")

#2.1 importar librerías necesarias


In [8]:
import os            #  trabajar sobre el sistema operativo
import sys           #  manipular archivos (cortar, copiar, borrar, crear)
import tarfile       #  Manipular archivos comprimidos (comprimir, descomprimir)
import time          #  calcular tiempo (en este caso tiempo de descarga de archivo)
import gzip          #  libreria para manipular archivos .gz
import shutil        #  Operaciones de archivo de alto nivel (copiar, borrar y archivar).
import struct        #  convertir entre cadenas de bytes y tipos de datos nativos Python(números y cadenas)
import pyprind
import numpy as np

ModuleNotFoundError: ignored

##2.2 variables con las urls de los sources y targets


In [0]:
#Nombre de la carpeta onde guardaremos los datasets
basepath='MNISTdatasets'
#Nombre de la carpeta en Google drive donde guardaremos el proyecto
googlepath='/content/drive/My Drive/allprojects/deep/manuscritos/'+basepath+'/'
# dataset de entrenamiento
trainSource='http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz'
trainTarget='train-images-idx3-ubyte.gz'
trainLabelSource='http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz'
trainLabelTarget='train-labels-idx1-ubyte.gz'
#  dataset para testing
testSource='http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz'
testTarget='test-images-idx3-ubyte.gz' 
testLabelSource='http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz'
testLabelTarget='test-labels-idx1-ubyte.gz'

##2.3 descargar los datasets (antes de ejecutar verificar que se tiene acceso a  Google drive)


In [8]:
download_dataset(trainSource,trainTarget,googlepath)
download_dataset(trainLabelSource,trainLabelTarget,googlepath)
download_dataset(testSource,testTarget,googlepath)
download_dataset(testLabelSource,testLabelTarget,googlepath)

The file /content/drive/My Drive/allprojects/deep/manuscritos/MNISTdatasets//train-images-idx3-ubyte.gz is downloaded
180% | 0 MB | 80.51 MB/s | 0 segundos transcurrido

##2.4 Extraemos todos los archivos del corpus  (primer método)


In [0]:
def write_file(filename, data):
  try:
    f = open(filename, "wb")
  except IOError as e:
    print (e.errno +"->"+ e.message)
  else:
    f.write(data)
    f.close()

def decompress(filename):
  if os.path.exists(filename):
    if filename.lower().endswith(".gz"):
      f = gzip.open(filename)
      write_file(filename[:filename.rfind(".gz")], f.read())
      f.close()
      print("extract finish->"+filename)
  else:
    print("No se ha encontrado ningún archivo.")

##2.4 Extraemos todos los archivos del corpus  (primer método)


In [10]:
decompress(googlepath+trainTarget)
decompress(googlepath+trainLabelTarget)
decompress(googlepath+testTarget)
decompress(googlepath+testLabelTarget)

extract finish->/content/drive/My Drive/allprojects/deep/manuscritos/MNISTdatasets/train-images-idx3-ubyte.gz
extract finish->/content/drive/My Drive/allprojects/deep/manuscritos/MNISTdatasets/train-labels-idx1-ubyte.gz
extract finish->/content/drive/My Drive/allprojects/deep/manuscritos/MNISTdatasets/test-images-idx3-ubyte.gz
extract finish->/content/drive/My Drive/allprojects/deep/manuscritos/MNISTdatasets/test-labels-idx1-ubyte.gz


##2.4 Extraemos todos los archivos del corpus  (segundo método)


In [12]:
#segundo metodo para descromprimir archivo gz
if (sys.version_info > (3, 0)):
    writemode = 'wb'
else:
    writemode = 'w'
#listamos los archivos .gz que se van a descomprimir
zipped_mnist = [f for f in os.listdir(googlepath) if f.endswith('ubyte.gz')]
for filename in zipped_mnist:
    filename=googlepath+filename
    with gzip.GzipFile(filename, mode='rb') as decompressed, open(filename[:-3], writemode) as outfile:
        outfile.write(decompressed.read())
        print("extract finish->"+filename)


extract finish->/content/drive/My Drive/allprojects/deep/manuscritos/MNISTdatasets/train-images-idx3-ubyte.gz
extract finish->/content/drive/My Drive/allprojects/deep/manuscritos/MNISTdatasets/train-labels-idx1-ubyte.gz
extract finish->/content/drive/My Drive/allprojects/deep/manuscritos/MNISTdatasets/test-images-idx3-ubyte.gz
extract finish->/content/drive/My Drive/allprojects/deep/manuscritos/MNISTdatasets/test-labels-idx1-ubyte.gz


# Paso 3

In [0]:
def load_mnist(path, kind='train'):
    # cargamos la data de MNIST desde el path de googledrive
    labels_path = os.path.join(path,'%s-labels-idx1-ubyte' % kind)
    images_path = os.path.join(path,'%s-images-idx3-ubyte' % kind)
    
    with open(labels_path, 'rb') as lbpath:
        # forma como leeremos las imagenes
        magic, n = struct.unpack('>II',lbpath.read(8))
        labels = np.fromfile(lbpath,dtype=np.uint8)

    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack(">IIII",imgpath.read(16))
        images = np.fromfile(imgpath,dtype=np.uint8).reshape(len(labels), 784)
        images = ((images / 255.) - .5) * 2
 
    return images, labels


##3.1 Visualizar parte de la data

In [14]:
X_train, y_train =load_mnist(googlepath, kind='train')
print('train--->filas: %d, columnas: %d' % (X_train.shape[0], X_train.shape[1]))

X_test, y_test = load_mnist(googlepath, kind='test')
print('Test--->filas: %d, columnas: %d' % (X_test.shape[0], X_test.shape[1]))

train--->filas: 60000, columnas: 784
Test--->filas: 10000, columnas: 784


##3.2 Aspectos de las imagenes

In [0]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=2, ncols=5, sharex=True, sharey=True,)
ax = ax.flatten()
for i in range(10):
    img = X_train[y_train == i][0].reshape(28, 28)
    ax[i].imshow(img, cmap='Greys')

ax[0].set_xticks([])
ax[0].set_yticks([])
plt.tight_layout()
plt.show()

## 3.3 Visualizando diferentes versiones del digito 7 

In [0]:
fig, ax = plt.subplots(nrows=5, ncols=5, sharex=True, sharey=True,)
ax = ax.flatten()
for i in range(25):
    img = X_train[y_train == 8][i].reshape(28, 28)
    ax[i].imshow(img, cmap='Greys')

ax[0].set_xticks([])
ax[0].set_yticks([])
plt.tight_layout()
plt.show()

##3.4Exportando objectos tipos numpy

In [0]:
np.savez_compressed(googlepath+'mnist_scaled.npz', 
                    X_train=X_train,
                    y_train=y_train,
                    X_test=X_test,
                    y_test=y_test)

# 4 Cargando archivos tipo numpy

In [18]:
mnist = np.load('/content/drive/My Drive/allprojects/deep/manuscritos/MNISTdatasets/mnist_scaled.npz')
mnist.files

['X_train', 'y_train', 'X_test', 'y_test']

## 4.1.1 Asignando variables

In [0]:
X_train,y_train,X_test,y_test=[mnist[f] for f in ['X_train','y_train','X_test','y_test']]
#eliminamos la variable para optimizar memoria
del mnist 
X_train.shape

#Paso 5. Implementar un RNA multicapa (MLP)

## Paso 5.1 

In [1]:
import os
#obtener el basepath por defecto
cwd = os.getcwd() 
print("basepath por defecto es: ", cwd)

os.chdir('/content/drive/My Drive/allprojects/deep/manuscritos')
cwd = os.getcwd() 
print(" El basepath por defecto ahora es: ", cwd)


basepath por defecto es:  /content
 El basepath por defecto ahora es:  /content/drive/My Drive/allprojects/deep/manuscritos


## Paso 5.2 configurar la RNA multicapa (MLP)

In [0]:
from neuralnet import NeuralNetMLP
n_epochs = 200
nn = NeuralNetMLP(n_hidden=100, 
                  l2=0.01, 
                  epochs=200, 
                  eta=0.0005,
                  minibatch_size=100, 
                  shuffle=True,
                  seed=1)

#paso 6. Entrenar la MLP

In [3]:
nn.fit(X_train=X_train[:55000], 
       y_train=y_train[:55000],
       X_valid=X_train[55000:],
       y_valid=y_train[55000:])

NameError: ignored